# Entity Framework Core Examples

Making use of notebooks to printout sql queries formed by EF
When in doubt, make use of notebook to create EF expression and print out the query string.
This can help when you need to optimize the query for performance.

Issues:
- "using var ctx..." gives error when "using" is used

In [1]:
#r "nuget: Microsoft.EntityFrameworkCore.SqlServer, 7.0.5"

using Microsoft.EntityFrameworkCore;
using System;

public class ItemContext : DbContext
{
	public DbSet<Item> Items { get; set; }
	public DbSet<Tag> Tags { get; set; }

	protected override void OnConfiguring(DbContextOptionsBuilder optionsBuilder)
	{
		// use a valid connection string here so the parser doesn't complain
		optionsBuilder.UseSqlServer("Server=(localdb)\\MSSQLLocalDB;Database=SomeDbName;Trusted_Connection=True");
	}
}

public class Item
{
	public int Id { get; set; }
	public string Name { get; set; } = string.Empty;
	public string Status { get; set; } = string.Empty;
	public int SomeId { get; set; }
	public DateTimeOffset Created { get; set; }
	public string EnumValue { get; set; } = string.Empty;
}

public class Tag
{
	public int Id { get; set; }
	public string Name { get; set; } = string.Empty;
	public string Value { get; set; } = string.Empty;
}

Installed Packages Microsoft.EntityFrameworkCore.SqlServer, 7.0.5

<br><br>

## SELECT 
Examples to show how EF forms SQL queries

In [2]:
var ctx = new ItemContext();
IQueryable<Item> query = ctx.Items;

// as long as we do not call ToList or FirstOrDefault
// the following LINQ will not attempt to establish
// connection to database defined in connection string

var itemId = 10;
query = ctx.Items.Where(item => item.Id > itemId);
Console.WriteLine(query.ToQueryString());

DECLARE @__itemId_0 int = 10;

SELECT [i].[Id], [i].[Created], [i].[EnumValue], [i].[Name], [i].[SomeId], [i].[Status]
FROM [Items] AS [i]
WHERE [i].[Id] > @__itemId_0


In [3]:
// when we use raw values in the expression, we do not get parameterized queries
// compare with the following:
query = ctx.Items.Where(item => item.Id > 10);
Console.WriteLine(query.ToQueryString());

SELECT [i].[Id], [i].[Created], [i].[EnumValue], [i].[Name], [i].[SomeId], [i].[Status]
FROM [Items] AS [i]
WHERE [i].[Id] > 10


In [4]:
var nameToMatch = "HELLO WORLD";
query = ctx.Items.Where(item => item.Name.ToUpper().Contains(nameToMatch));
Console.WriteLine(query.ToQueryString());

DECLARE @__nameToMatch_0 nvarchar(4000) = N'HELLO WORLD';

SELECT [i].[Id], [i].[Created], [i].[EnumValue], [i].[Name], [i].[SomeId], [i].[Status]
FROM [Items] AS [i]
WHERE (@__nameToMatch_0 LIKE N'') OR CHARINDEX(@__nameToMatch_0, UPPER([i].[Name])) > 0


In [5]:
query = ctx.Items.Where(item => item.Created < DateTime.Now && item.Id == itemId);
Console.WriteLine(query.ToQueryString());

DECLARE @___itemId_0 int = 10;

SELECT [i].[Id], [i].[Created], [i].[EnumValue], [i].[Name], [i].[SomeId], [i].[Status]
FROM [Items] AS [i]
WHERE [i].[Created] < CAST(GETDATE() AS datetimeoffset) AND [i].[Id] = @___itemId_0


In [6]:
// break up Where clauses -> doesn't seem to affect SQL generated
query = ctx.Items.Where(item => item.Created < DateTime.Now).Where(item => item.Id == 300);
Console.WriteLine(query.ToQueryString());

SELECT [i].[Id], [i].[Created], [i].[EnumValue], [i].[Name], [i].[SomeId], [i].[Status]
FROM [Items] AS [i]
WHERE [i].[Created] < CAST(GETDATE() AS datetimeoffset) AND [i].[Id] = 300


This example shows why it's a bad idea to use a large collection with Contains(...)

In [7]:
var list = Enumerable.Range(1, 1000).Select(x => x.ToString());
query = ctx.Items.Where(item => list.Contains(item.Name));
Console.WriteLine(query.ToQueryString());

SELECT [i].[Id], [i].[Created], [i].[EnumValue], [i].[Name], [i].[SomeId], [i].[Status]
FROM [Items] AS [i]
WHERE [i].[Name] IN (N'1', N'2', N'3', N'4', N'5', N'6', N'7', N'8', N'9', N'10', N'11', N'12', N'13', N'14', N'15', N'16', N'17', N'18', N'19', N'20', N'21', N'22', N'23', N'24', N'25', N'26', N'27', N'28', N'29', N'30', N'31', N'32', N'33', N'34', N'35', N'36', N'37', N'38', N'39', N'40', N'41', N'42', N'43', N'44', N'45', N'46', N'47', N'48', N'49', N'50', N'51', N'52', N'53', N'54', N'55', N'56', N'57', N'58', N'59', N'60', N'61', N'62', N'63', N'64', N'65', N'66', N'67', N'68', N'69', N'70', N'71', N'72', N'73', N'74', N'75', N'76', N'77', N'78', N'79', N'80', N'81', N'82', N'83', N'84', N'85', N'86', N'87', N'88', N'89', N'90', N'91', N'92', N'93', N'94', N'95', N'96', N'97', N'98', N'99', N'100', N'101', N'102', N'103', N'104', N'105', N'106', N'107', N'108', N'109', N'110', N'111', N'112', N'113', N'114', N'115', N'116', N'117', N'118', N'119', N'120', N'121', N'122', N'1

In [8]:
// test it out before using in prod
// this might be an improvement because the list is parameterized
// see https://stackoverflow.com/questions/58883887/in-operator-with-a-very-large-list-is-too-slow-in-sql-server-what-should-i-use?rq=3
var list = Enumerable.Range(1, 1000).Select(x => x.ToString());
var numbers = System.Text.Json.JsonSerializer.Serialize(list);
query = ctx.Items
  .FromSql($"SELECT * FROM [Items] where [Items].Name IN (SELECT VALUE FROM OPENJSON({numbers}))");
Console.WriteLine(query.ToQueryString());

DECLARE p0 nvarchar(max) = N'["1","2","3","4","5","6","7","8","9","10","11","12","13","14","15","16","17","18","19","20","21","22","23","24","25","26","27","28","29","30","31","32","33","34","35","36","37","38","39","40","41","42","43","44","45","46","47","48","49","50","51","52","53","54","55","56","57","58","59","60","61","62","63","64","65","66","67","68","69","70","71","72","73","74","75","76","77","78","79","80","81","82","83","84","85","86","87","88","89","90","91","92","93","94","95","96","97","98","99","100","101","102","103","104","105","106","107","108","109","110","111","112","113","114","115","116","117","118","119","120","121","122","123","124","125","126","127","128","129","130","131","132","133","134","135","136","137","138","139","140","141","142","143","144","145","146","147","148","149","150","151","152","153","154","155","156","157","158","159","160","161","162","163","164","165","166","167","168","169","170","171","172","173","174","175","176","177","178","179","180

<br><br>

## SELECT + DISTINCT + ORDER BY

In [9]:
// if we have logging framework added, entity framework
// will complain about the calling order of Distinct + OrderBy
// the generated query will not show "ORDER BY"
var wrongQuery = ctx.Items
    .Select(item => new { item.Name, item.Created })
    .OrderBy(it => it.Created)
    .Distinct();

Console.WriteLine("Wrong ORDER BY + DISTINCT -- the ORDER BY is missing in SQL query");
Console.WriteLine(wrongQuery.ToQueryString());
Console.WriteLine();

// the SQL generated will show "ORDER BY" but
// as of Mar 2023, the generated SQL doesn't seem to be "right"
// the following would look more "right"
// SELECT DISTINCT ...
// FROM TABLE
// ORDER BY Created
var correctQuery = ctx.Items
    .Select(item => new { item.Name, item.Created })
    .Distinct()
    .OrderBy(it => it.Created);

Console.WriteLine("Correct DISTINCT + ORDER BY -- looks more complicated but ORDER BY is observed in SQL query");
Console.WriteLine(correctQuery.ToQueryString());


Wrong ORDER BY + DISTINCT -- the ORDER BY is missing in SQL query
SELECT DISTINCT [i].[Name], [i].[Created]
FROM [Items] AS [i]

Correct DISTINCT + ORDER BY -- looks more complicated but ORDER BY is observed in SQL query
SELECT [t].[Name], [t].[Created]
FROM (
    SELECT DISTINCT [i].[Name], [i].[Created]
    FROM [Items] AS [i]
) AS [t]
ORDER BY [t].[Created]


In [10]:
// this query demonstrate query that requires a custom order by
// make sure to double check query generated
// there used to be a bug with how the query was generated in
// earlier versions of entity framework
var query = ctx.Items
    .Select(item => new
    {
        item,
        sortIndex =
            item.EnumValue == "Pending" ? 0 :
            item.EnumValue == "Accepted" ? 1 :
            item.EnumValue == "Rejected" ? 2 : 3
    })
    .OrderBy(anon => anon.sortIndex)
    .Select(anon => anon.item);

Console.WriteLine(query.ToQueryString());

SELECT [i].[Id], [i].[Created], [i].[EnumValue], [i].[Name], [i].[SomeId], [i].[Status]
FROM [Items] AS [i]
ORDER BY CASE
    WHEN [i].[EnumValue] = N'Pending' THEN 0
    WHEN [i].[EnumValue] = N'Accepted' THEN 1
    WHEN [i].[EnumValue] = N'Rejected' THEN 2
    ELSE 3
END


<br><br>

## SKIP + TAKE

In [11]:
// EF parameterized the raw values 100/1000
var query = ctx.Items.Skip(100).Take(1000);
Console.WriteLine(query.ToQueryString());

DECLARE @__p_0 int = 100;
DECLARE @__p_1 int = 1000;

SELECT [i].[Id], [i].[Created], [i].[EnumValue], [i].[Name], [i].[SomeId], [i].[Status]
FROM [Items] AS [i]
ORDER BY (SELECT 1)
OFFSET @__p_0 ROWS FETCH NEXT @__p_1 ROWS ONLY


<br><br>

## JOIN

In [12]:
// example of left join
var joinQuery = from item in ctx.Items
                join tag in ctx.Tags
                on item.Name equals tag.Name into tagged
                from t in tagged.DefaultIfEmpty()
                select new { item.Name, t.Value };

Console.WriteLine(joinQuery.ToQueryString());

SELECT [i].[Name], [t].[Value]
FROM [Items] AS [i]
LEFT JOIN [Tags] AS [t] ON [i].[Name] = [t].[Name]
